# Notebook 07: Targeted Adversarial Robustness Testing (The extreme limits now)

This notebook implements a focused set of adversarial experiments to evaluate the robustness limits of the FL-IDS system under strategically crafted attack scenarios. It builds on previous work in Notebook 06 and stress-tests the median, Krum, and drift-aware aggregation strategies through precision attacks that bypass typical detection.

## Objectives

- Evaluate the failure threshold of **Median Aggregation** under progressive majority collusion scenarios.
- Test **Gradient Positioning Attacks** designed to influence aggregation while remaining statistically benign.
- Simulate **Multi-Vector Attacks**, combining multiple attack types such as model poisoning, label flipping, and timing interference.
- Bypass **Drift Detection Mechanisms** (KS-test, PSI) using statistical camouflage.

These experiments aim to uncover subtle vulnerabilities and help define the true operational limits of your deployed FL-IDS system, especially under real-world edge constraints like Raspberry Pi devices.



In [2]:
import os
import joblib
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import f1_score, precision_score, recall_score

# === Paths ===
base_path = "D:/August-Thesis/FL-IDS-Surveillance"
results_path = os.path.join(base_path, "notebooks/results")
model_path = os.path.join(results_path, "models/unsupervised/federated/final_federated_autoencoder_20rounds.h5")
scaler_path = os.path.join(results_path, "scalers/minmax_scaler_client_3.pkl")
data_path = os.path.join(base_path, "data/processed/federated/unsupervised")
test_path = os.path.join(base_path, "data/processed/surv_unsupervised/test_mixed.csv")

# === Loading the Global Model ===
base_autoencoder = load_model(model_path, compile=False)
base_autoencoder.compile(optimizer="adam", loss=MeanSquaredError())

# === the Scaler ===
minmax_scaler = joblib.load(scaler_path)

# === Load Client Data ===
client_ids = [f"client_{i}" for i in range(1, 6)]
client_data = {
    cid: pd.read_csv(os.path.join(data_path, cid, "train.csv"))
    for cid in client_ids
}

# === Load Test Data ===
test_df = pd.read_csv(test_path, low_memory=False)
feature_cols = list(minmax_scaler.feature_names_in_)
X_test_ae_scaled = minmax_scaler.transform(test_df[feature_cols])
y_test_unsup = test_df["Attack_label"].values

print("Setup complete:")
print(f"- Loaded model: {os.path.basename(model_path)}")
print(f"- Clients: {client_ids}")
print(f"- Test samples: {X_test_ae_scaled.shape[0]}")


Setup complete:
- Loaded model: final_federated_autoencoder_20rounds.h5
- Clients: ['client_1', 'client_2', 'client_3', 'client_4', 'client_5']
- Test samples: 2218834


In [3]:
from sklearn.metrics import f1_score, precision_score, recall_score

threshold = 0.000639

# === Predict with global model on test set ===
reconstructed = base_autoencoder.predict(X_test_ae_scaled, verbose=0)
reconstruction_errors = np.mean(np.square(X_test_ae_scaled - reconstructed), axis=1)

# === Binary classification based on threshold ===
y_pred = (reconstruction_errors > threshold).astype(int)

# === Metrics ===
f1 = f1_score(y_test_unsup, y_pred)
precision = precision_score(y_test_unsup, y_pred)
recall = recall_score(y_test_unsup, y_pred)

print("=== Baseline Performance ===")
print(f"F1 Score:     {f1:.4f}")
print(f"Precision:    {precision:.4f}")
print(f"Recall:       {recall:.4f}")


=== Baseline Performance ===
F1 Score:     0.8373
Precision:    0.9877
Recall:       0.7266
